In [ ]:
import pandas as pd
from os.path import dirname, isfile, join

# read simulation scenarios
mdir = r"../../3_models/sfincs"
rdir = r"../../4_results"
scens = pd.read_csv(join(rdir, 'sim_SCEN_all_rps7.csv'), index_col=0)
scens2 = scens.copy()
scens2['scen'] = [f'{scen}_dt0' for scen in scens['scen']]
scens = pd.concat([scens, scens2]).rename(columns={'h_tsw_rp': 'h_rp'})
# scens = pd.concat([
#     pd.read_csv(join(rdir, 'sim_SCEN_all_rps7.csv'), index_col=0),
#     pd.read_csv(join(rdir, 'sim_SCEN.csv'), index_col=0),
#     pd.read_csv(join(rdir, 'sim_SCEN_fulldep1.csv'), index_col=0),
#     pd.read_csv(join(rdir, 'sim_SCEN_indep1.csv'), index_col=0),
# ], axis=0).rename(columns={'h_tsw_rp': 'h_rp'}).drop(columns='count').drop_duplicates()
scens['finished'] = False
scens

In [ ]:
# check model simulation status
import subprocess
import os
import glob
from run_sfincs import postprocess
from datetime import datetime, timedelta

def check_finished(root):
    finished = False
    inprogress = False
    failed = False
    v295 = False
    fn_log = join(root, 'sfincs.log')
    t = 0.0
    if not isfile(join(root, 'sfincs.inp')):
        print(f'MISSING: {root}')
        return False, inprogress, t
    # if isfile(join(root, 'gis', 'hmax.tif')):
    #     return True, inprogress, t
    if isfile(fn_log):
        mtime = os.path.getmtime(fn_log)
        mtime = datetime.fromtimestamp(mtime)
        mtime_str = mtime.strftime("%H:%M:%S")
        dt = datetime.now() - mtime
        with open(fn_log, 'r') as f:
            lines = f.readlines()
            if len(lines) == 0:
                print(f'RUNNING  UNIX: {root} {mtime_str}')
                inprogress = True
                return finished, inprogress, t
            for l in lines:
                # if 'Rev: 295' in l:
                #     v295 = True
                if 'Simulation stopped' in l:
                    print(f'FAILED: {root}')
                    failed = True
                    break
                elif 'Simulation is finished' in l:
                    finished = True
                elif 'Total run time' in l:
                    t = float(l.split(':')[1].strip())
                    break
        if not finished and dt > timedelta(seconds=3600):
            print(f'OVERDUE: {root}')
            failed = True
        if failed:
            os.rename(fn_log, join(root, 'sfincs.log.failed'))
        elif finished and isfile(join(root, 'zsmax.dat')):
            print(f'postprocessing {root}')
            postprocess(root)
        elif not finished:
            print(f'RUNNING WINDOWS: {root} {mtime_str}')
            inprogress = True
    if finished and v295:
        os.rename(fn_log, join(root, 'sfincs.log.org'))
        if isfile(join(root, 'gis', 'hmax.tif')):
            os.rename(join(root, 'gis', 'hmax.tif'), join(root, 'gis', 'hmax_org.tif'))
        if isfile(join(root, 'fig', 'hmax.png')):
            os.rename(join(root, 'fig', 'hmax.png'), join(root, 'fig', 'hmax_org.png'))
        finished = False
    return finished, inprogress, t

# runs = [dirname(fn) for fn in glob.glob(join(mdir, '*', 'sfincs.inp')) if not check_finished(dirname(fn))]
runs = []
for i, row in scens.iterrows():
    if row['finished']:
        runs.append((True, False, row['duration']))
    else:
        runs.append(check_finished(join(mdir, row['scen'])))

finished, inprogress, duration = zip(*runs)

scens['finished'] = finished
scens['inprogress'] = inprogress
scens['duration'] = duration
# scens.head()

In [ ]:
import numpy as np
from datetime import datetime, timedelta

mean_duration = np.array(list(duration))[np.array(list(finished))].mean()
todo = len(runs)-sum(finished)
ed = todo*mean_duration/sum(inprogress)
eta = (datetime.now() + timedelta(seconds=ed)).strftime('%d-%m %H:%M')

print(f'finished: {sum(finished)}/{len(runs)}')
print(f'left to do: {todo}')
print(f'in progress: {sum(inprogress)}')
print(f'mean simulation time: {mean_duration:.2f} sec')
print(f'ETA: {eta}')
